In [ ]:
!pip install gdown -qq
!pip install git+https://github.com/tensorflow/docs
# !gdown --id 11eFE_quM2_2-h3H_zTTjq0i0D6pkx62Z
# !unzip ../input/wlasl-processed -d ./WLASL2000
# !gdown --id 13lZfwNZcTm6BVnUU0sqyrzwUL-dyaCos
# https://drive.google.com/file/d/13lZfwNZcTm6BVnUU0sqyrzwUL-dyaCos/view?usp=sharing

In [ ]:
import os
import re
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
import shutil
import random
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from keras import Sequential, Input, layers
from keras.regularizers import l2
from keras.layers import Dense, BatchNormalization, ReLU, Dropout, MaxPooling2D, Conv2D, Flatten, BatchNormalization, Dropout
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
import cv2 as cv
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3
path = './'
id3 = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"

In [ ]:
path2 = "../input/wlasl-complete/WLASL2000"
videos = os.listdir(path2)
print(len(videos))
wlas_df = pd.read_json("../input/wlasl-processed/WLASL_v0.3.json")
wlas_df.head()
wlas_df['instances'][0][:2]
def get_features(feat) :
    ids = []
    urls = []
    for f in feat :
        id = f['video_id']
        url = f['url']
#         print(id)
        if os.path.exists(f"{path2}/{id}.mp4") :
            ids.append(id)
            urls.append(url)
    return ids,urls

def get_video_ids(feats) :
    videos = []
    for feat in feats :
        video_id = feat['video_id']
#         print(video_id)
        if os.path.exists(f"{path2}/{video_id}.mp4") :
            videos.append(video_id)
    return videos

df = pd.DataFrame(columns=['gloss','video_id','url'])
wlas_df['videos_ids'] = wlas_df['instances'].apply(get_video_ids)
wlas_df
for row in tqdm(wlas_df.iterrows()) :
    ids,urls = get_features(row[1][1])
    words = [row[1][0]]*len(ids)
    df = df.append(pd.DataFrame(list(zip(words,ids,urls)),
                   columns=df.columns),ignore_index=True)
df.to_csv('feature.csv', index=False)
wlas_df.to_csv('wlas_df.csv',index=False)
print(df.shape)
df.head()

def extend_str(s,count=5) :
    s = str(s)
    s = (count-len(s))*"0"+s
    return s

df['video_id'] = df['video_id'].apply(extend_str)
# frame_count = []
# for video_id in tqdm(df['video_id']) :
#     video = cv.VideoCapture(f"{path2}/{video_id}.mp4")
#     length = int(video.get(cv.CAP_PROP_FRAME_COUNT))
#     frame_count.append(length)
# #     break

# df['frame_count'] = frame_count
# print(df['frame_count'].max(),df['frame_count'].min())
# sns.histplot(df['frame_count'])
# plt.show()
df.head()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
df['type'] = 0
for i,(train_idx,val_idx) in enumerate(skf.split(df,df['gloss'])) :
    if i == 0 :
        df.loc[train_idx,'type'] = 'train'
        df.loc[val_idx,'type'] = 'val' 
    elif i == 1 :
        df.loc[val_idx,'type'] = 'test'
    else :
        break
        
paths_arr=['train','test','val']
for i in paths_arr :
    if not os.path.exists(f"{path}/{i}") :
        os.makedirs(f'{path}/{i}')

obs = df['gloss'].unique().tolist()
for i in obs :
    for j in paths_arr :
        if not os.path.exists(f"{path}/{j}/{i}") :
            os.makedirs(f'{path}/{j}/{i}')

for row in tqdm(df.iterrows()) :
    type_type = row[1].type
    vid_id = row[1].video_id
    gloss = row[1].gloss
    shutil.copy(f"{path2}/{vid_id}.mp4",f"{path}/{type_type}/{gloss}/{vid_id}.mp4")
# !zip -r train.zip "./train" 
# !zip -r test.zip "./test" 
# !zip -r val.zip "./val" 

<a href="./train.zip"> Download train File </a>
<a href="./test.zip"> Download test File </a>
<a href="./val.zip"> Download val File </a>

In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=25)
    return embed.embed_file('./animation.gif')


i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']
i3d
sample_video = load_video(f"{path}/{df.loc[0,'type']}/{df.loc[0,'gloss']}/{df.loc[0,'video_id']}.mp4",64)
sample_video.shape

In [ ]:
def predict(sample_video):
  # Add a batch axis to the sample video.
    model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

    logits = i3d(model_input)['default'][0]
    probabilities = tf.nn.softmax(logits)

    print("Top 5 actions:")
    for i in np.argsort(probabilities)[::-1][:5]:
        print(f"  {obs[i]}: {probabilities[i] * 100:5.2f}%")
predict(sample_video)
to_gif(sample_video)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(obs)
le.transform(obs)

In [ ]:
# feature_extractor_model = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
# BATCH_SIZE=32
# NO_FRAMES=50
# IMG_WIDTH=224
# IMG_HEIGHT=224
# CHANNELS=3
# n_classes = len(obs)
# input_shape = (NO_FRAMES,IMG_WIDTH,IMG_HEIGHT,CHANNELS)
# feature_extractor_layer = hub.KerasLayer(
#         feature_extractor_model,
#         input_shape=input_shape,
#         trainable=False)


# def crop_center_square(frame):
#     y, x = frame.shape[0:2]
#     min_dim = min(y, x)
#     start_x = (x // 2) - (min_dim // 2)
#     start_y = (y // 2) - (min_dim // 2)
#     return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

# def load_video(path, max_frames=NO_FRAMES, resize=(IMG_WIDTH, IMG_HEIGHT)):
#     print(path,max_frames)
#     cap = cv2.VideoCapture(path)
#     frames = []
#     try:
#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frame = crop_center_square(frame)
#             frame = cv2.resize(frame, resize)
#             frame = frame[:, :, [2, 1, 0]]
#             frames.append(frame)

#             if len(frames) == max_frames:
#                 break
#     finally:
#         cap.release()
#     return np.array(frames) / 255.0

# def to_gif(images):
#     converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
#     imageio.mimsave('./animation.gif', converted_images, fps=25)
#     return embed.embed_file('./animation.gif')

# model = Sequential([
# #         Input((10, 224, 224, 3,)),
#         feature_extractor_layer,
#         Dense(n_classes, activation='softmax')
#     ])
# model.summary()


# ds_train = df[df['type']=='train'][['gloss','video_id']]
# ds_train['video_id'] = ds_train['video_id'].apply(lambda x:float(x))
# ds_test = df[df['type']=='test'][['gloss','video_id']]
# ds_test['video_id'] = ds_test['video_id'].apply(lambda x:float(x))
# ds_val = df[df['type']=='val'][['gloss','video_id']]
# ds_val['video_id'] = ds_val['video_id'].apply(lambda x:float(x))
# ds_train = tf.data.Dataset.from_tensor_slices((ds_train['video_id'].values, le.transform(ds_train['gloss'].values)))
# ds_test = tf.data.Dataset.from_tensor_slices((ds_test['video_id'].values, le.transform(ds_test['gloss'].values)))
# ds_val = tf.data.Dataset.from_tensor_slices((ds_val['video_id'].values, le.transform(ds_val['gloss'].values)))
# # ds_train = ds_train.map(load_video).batch(2)
# print(ds_train)
# for i in ds_train:
#     print(i)
#     break
    
# def load_video(path1,y, max_frames=NO_FRAMES, resize=(IMG_WIDTH, IMG_HEIGHT)):
# #     print(path1)
#     path = f"../input/wlasl-complete/WLASL2000/{extend_str(str(int(path1.numpy())))}.mp4"
#     cap = cv2.VideoCapture(path)
#     frames = np.zeros((max_frames,IMG_WIDTH,IMG_HEIGHT,CHANNELS))
#     i = 0
#     try:
#         while True:
#             ret, frame = cap.read()
            
#             if not ret:
#                 break
#             frame = crop_center_square(frame)
#             frame = cv2.resize(frame, resize)
#             frame = frame[:, :, [2, 1, 0]]
#             frames[i] = frame
#             i+=1

#             if i == max_frames:
#                 break
#     finally:
#         cap.release()
# #     print(len(frames))
#     return tf.constant(frames)/255.0,y

# def process_video(x,y) :
#     x,y=tf.py_function(load_video, [x,y], [tf.float64, tf.int64,])
#     return x,y
    
    

# # ds_train = ds_train.map(lambda x,y: tf.py_function(load_video, [x,y], [tf.float64, tf.int64,])).batch(BATCH_SIZE)
# ds_train = ds_train.map(process_video).batch(BATCH_SIZE)
# # ds_test = ds_test.map(lambda x,y: tf.py_function(load_video, [x,y], [tf.float64, tf.int64,])).batch(BATCH_SIZE)
# ds_test = ds_test.map(process_video).batch(BATCH_SIZE)
# # ds_val = ds_val.map(lambda x,y: tf.py_function(load_video, [x,y], [tf.float64, tf.int64,])).batch(BATCH_SIZE)
# ds_val = ds_val.map(process_video).batch(BATCH_SIZE)
# ds_train

In [ ]:
feature_extractor_model = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
BATCH_SIZE=32
SHUFFLE_WINDOW=32
NO_FRAMES=50
IMG_WIDTH=224
IMG_HEIGHT=224
CHANNELS=3
n_classes = len(obs)
input_shape = (NO_FRAMES,IMG_WIDTH,IMG_HEIGHT,CHANNELS)
feature_extractor_layer = hub.KerasLayer(
        feature_extractor_model,
        input_shape=input_shape,
        trainable=False)


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=NO_FRAMES, resize=(IMG_WIDTH, IMG_HEIGHT)):
    print(path,max_frames)
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

def to_gif(images):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave('./animation.gif', converted_images, fps=25)
    return embed.embed_file('./animation.gif')

model = Sequential([
#         Input((10, 224, 224, 3,)),
        feature_extractor_layer,
        Dense(n_classes, activation='softmax')
    ])
model.summary()

from sklearn.utils import shuffle
df = shuffle(df,random_state=42)
ds_train = df[df['type']=='train'][['gloss','video_id']]
ds_train['video_id'] = ds_train['video_id'].apply(lambda x:float(x))
ds_test = df[df['type']=='test'][['gloss','video_id']]
ds_test['video_id'] = ds_test['video_id'].apply(lambda x:float(x))
ds_val = df[df['type']=='val'][['gloss','video_id']]
ds_val['video_id'] = ds_val['video_id'].apply(lambda x:float(x))
ds_x_train = tf.constant(ds_train['video_id'].values)
ds_y_train = tf.constant(le.transform(ds_train['gloss'].values))
# ds_y_train = tf.constant(ds_train['gloss'].values)

ds_x_test = tf.constant(ds_test['video_id'].values)
ds_y_test = tf.constant(le.transform(ds_test['gloss'].values))
# ds_y_test = tf.constant(ds_test['gloss'].values)

ds_x_val = tf.constant(ds_val['video_id'].values)
ds_y_val = tf.constant(le.transform(ds_val['gloss'].values))
# ds_y_val = tf.constant(ds_val['gloss'].values)

# ds_train = ds_train.map(load_video).batch(2)

    
def load_video(path1,y, max_frames=NO_FRAMES, resize=(IMG_WIDTH, IMG_HEIGHT)):
#     print(path1)
#     path = f"../input/wlasl-complete/WLASL2000/{extend_str(str(int(path1.numpy())))}.mp4"
    path = f"../input/wlasl-complete/WLASL2000/{extend_str(str(int(path1)))}.mp4"
    cap = cv2.VideoCapture(path)
    frames = np.zeros((max_frames,IMG_WIDTH,IMG_HEIGHT,CHANNELS))
    i = 0
    try:
        while True:
            ret, frame = cap.read()
            
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames[i] = frame
            i+=1

            if i == max_frames:
                break
    finally:
        cap.release()
#     print(len(frames))
    return tf.constant(frames)/255.0,y


def gen(x,y):
    for i,j in enumerate(x) :
#         print(i,j)
        x1,y1 = load_video(j,y[i])
        yield x1,y1

ds_train = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        tf.TensorSpec(shape=(NO_FRAMES,IMG_WIDTH,IMG_HEIGHT,CHANNELS), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.int64)),
    args=(ds_x_train, ds_y_train)
).shuffle(SHUFFLE_WINDOW).batch(BATCH_SIZE)

ds_test = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        tf.TensorSpec(shape=(NO_FRAMES,IMG_WIDTH,IMG_HEIGHT,CHANNELS), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.int64)),
    args=(ds_x_test, ds_y_test)
).shuffle(SHUFFLE_WINDOW).batch(BATCH_SIZE)

ds_val = tf.data.Dataset.from_generator(
    gen,
    output_signature=(
        tf.TensorSpec(shape=(NO_FRAMES,IMG_WIDTH,IMG_HEIGHT,CHANNELS), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.int64)),
    args=(ds_x_val, ds_y_val)
).shuffle(SHUFFLE_WINDOW).batch(BATCH_SIZE)
ds_train

In [ ]:
print(ds_train.element_spec)
for image, label in ds_train.take(1):
    print("Image shape: ", image.shape)
    print("Label: ", label.numpy())
print("Number of samples in train: ", ds_train.cardinality().numpy()," in test: ",ds_test.cardinality().numpy())
tf.data.experimental.UNKNOWN_CARDINALITY

In [ ]:
TF_NUM_EPOCHS=2
model.compile(optimizer=Adam(learning_rate=3e-4),
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(ds_train,
                    validation_data=ds_val,
                    epochs=TF_NUM_EPOCHS, batch_size=BATCH_SIZE, verbose=1)